# Análise exploratória de dados Pub Med e Tweet Covid19

## Instalação dos pacotes necessários para EDA

In [ ]:
# Executar esse bloco caso não tenha o pacote instalado
# install.packages("tidyverse")
# install.packages("magrittr")
# install.packages("tm")
# install.packages("wordcloud")

## Carregando os pacotes

In [1]:
library(wordcloud)
library(magrittr)
library(tm)
library(tidyverse)
library(tidyr)
library(cluster)

Loading required package: RColorBrewer

Loading required package: NLP

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::annotate() masks NLP::annotate()
✖ tidyr::extract()    masks magrittr::extract()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::set_names()  masks magrittr::set_names()



## Leitura dos dados Pub Med

In [ ]:
# Leitura do arquivo por linha
covid <- readLines("PM-COVID-4064-TM.txt")

In [ ]:
# Leitura do arquivo por DF
covid_df <- read.delim("PM-COVID-4064-TM.txt", header = FALSE, sep = "\n")
covid_df$doc_id <- seq.int(nrow(covid_df))
covid_df <- covid_df[,c(2,1)]
colnames(covid_df) <- c("doc_id", "text")

In [ ]:
covid_source <- VectorSource(covid)
coviddf_source <- DataframeSource(covid_df)

In [ ]:
covid_corpus <- VCorpus(covid_source)
coviddf_corpus <- VCorpus(coviddf_source)

In [ ]:
print(covid_corpus)
print(coviddf_corpus)
covid_corpus[[1]]
covid_corpus[[1]]$content

## Limpeza dos dados Pub Med

In [ ]:
# Exemplo
tolower(covid_corpus[[1]]$content)
toupper(covid_corpus[[1]]$content)
removePunctuation(covid_corpus[[1]]$content)
removeNumbers(covid_corpus[[1]]$content)
stripWhitespace(covid_corpus[[1]]$content)

In [ ]:
# Lista de stopwords en, pt e es
stopwords("en"); stopwords("pt"); stopwords("es")

In [60]:
new_stops <- c("COVID", "the", "The", stopwords("en"))

In [ ]:
# Exemplo
removeWords(covid_corpus[[1]]$content, stopwords("en")) %>% stripWhitespace()
removeWords(covid_corpus[[1]]$content, new_stops) %>% stripWhitespace()

## Aplicando a limpeza e normalização dos dados em toda base Pub Med

In [ ]:
covid_clean <- tm_map(covid_corpus, removeWords, words = c(new_stops))
covid_clean <- tm_map(covid_clean, removePunctuation)
covid_clean[[1]]$content
covid_clean <- tm_map(covid_clean, stripWhitespace)

## Frequência dos Termos Pub Med 

In [ ]:
covid_dtm <- DocumentTermMatrix(covid_clean)
covid_dtm
covid_m1 <- as.matrix(covid_dtm)

In [ ]:
covid_tdm <- TermDocumentMatrix(covid_clean)
covid_tdm
covid_m2 <- as.matrix(covid_tdm)

In [ ]:
covid_m1[1:10, 1000:1010]
covid_m2[1000:1010, 1:10]

In [ ]:
term_frequency_covid <- rowSums(covid_m2)
term_frequency_covid <- sort(term_frequency_covid, decreasing = TRUE)
head(term_frequency_covid, 100) 
term_frequency_covid[100:200]

## Visualização de Dados Pub Med

In [ ]:
term <- names(term_frequency_covid)
num <- term_frequency_covid
word_freqs <- data.frame(term, num)
wordcloud(word_freqs$term, word_freqs$num, max.words = 100, colors = "red")

In [ ]:
freq <- sort(colSums(as.matrix(covid_dtm)), decreasing=TRUE)   
barplot(freq[1:10],col="lightgreen",las=2)

In [ ]:
dtmss <- removeSparseTerms(covid_dtm, 0.80)   
dtmss   
d <- dist(t(dtmss), method="euclidian")   
fit <- hclust(d=d, method="complete")
fit 
plot(fit, hang=1) 
groups <- cutree(fit, k=6)   
rect.hclust(fit, k=6, border="red")

In [ ]:
d <- dist(t(dtmss), method="euclidian")   
kfit <- kmeans(d, 2)   
clusplot(as.matrix(d), kfit$cluster, color=T, shade=T, labels=2, lines=0)

In [ ]:
set.seed(1234)
wordcloud(words = word_freqs$term, freq = word_freqs$num, min.freq = 1,
          max.words=200, random.order=FALSE, rot.per=0.35, 
          colors=brewer.pal(8, "Dark2"))

## Leitura dos dados Tweet Covid19

In [43]:
tweets_covid <-read_csv('covid19_tweets.csv')

Parsed with column specification:
cols(
  user_name = col_character(),
  user_location = col_character(),
  user_description = col_character(),
  user_created = col_datetime(format = ""),
  user_followers = col_double(),
  user_friends = col_double(),
  user_favourites = col_double(),
  user_verified = col_logical(),
  date = col_datetime(format = ""),
  text = col_character(),
  hashtags = col_character(),
  source = col_character(),
  is_retweet = col_logical()
)



In [44]:
glimpse(tweets_covid)

Rows: 179,108
Columns: 13
$ user_name        <chr> "ᏉᎥ☻լꂅϮ", "Tom Basile 🇺🇸", "Time4fisticuffs", "ethel…
$ user_location    <chr> "astroworld", "New York, NY", "Pewee Valley, KY", "S…
$ user_description <chr> "wednesday addams as a disney princess keepin it [̲̅…
$ user_created     <dttm> 2017-05-26 05:46:42, 2009-04-16 20:06:23, 2009-02-2…
$ user_followers   <dbl> 624, 2253, 9275, 197, 101009, 1180, 79956, 608, 25, …
$ user_friends     <dbl> 950, 1677, 9525, 987, 168, 1071, 54810, 355, 29, 342…
$ user_favourites  <dbl> 18775, 24, 7254, 1488, 101, 1287, 3801, 95, 18, 2980…
$ user_verified    <lgl> FALSE, TRUE, FALSE, FALSE, FALSE, FALSE, FALSE, FALS…
$ date             <dttm> 2020-07-25 12:27:21, 2020-07-25 12:27:17, 2020-07-2…
$ text             <chr> "If I smelled the scent of hand sanitizers today on …
$ hashtags         <chr> NA, NA, "['COVID19']", "['COVID19']", "['CoronaVirus…
$ source           <chr> "Twitter for iPhone", "Twitter for Android", "Twitte…
$ is_retweet       <lgl> F

In [45]:
tweets_covid<-tweets_covid %>%select(c(user_location, date, text, source))

In [46]:
tweets_covid%>%head(5)

user_location,date,text,source
<chr>,<dttm>,<chr>,<chr>
astroworld,2020-07-25 12:27:21,"If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0",Twitter for iPhone
"New York, NY",2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it have made more sense to have the players pay their respects to the A… https://t.co/1QvW0zgyPu,Twitter for Android
"Pewee Valley, KY",2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump never once claimed #COVID19 was a hoax. We all claim that this effort to… https://t.co/Jkk8vHWHb3,Twitter for Android
Stuck in the Middle,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me is an appreciation for the simple things that were always around me… https://t.co/Z0pOAlFXcW,Twitter for iPhone
Jammu and Kashmir,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirusUpdates #COVID19 @kansalrohit69 @DrSyedSehrish @airnewsalerts @ANI… https://t.co/MN0EEcsJHh,Twitter for Android


In [47]:
tweets_covid%>%tail(5)

user_location,date,text,source
<chr>,<dttm>,<chr>,<chr>
"Ilorin, Nigeria",2020-08-29 19:44:21,"Thanks @IamOhmai for nominating me for the @WHO #WearAMask challenge. I nominate @abdlbaasit_ @hvbxxb,… https://t.co/IegQDkKudT",Twitter for Android
Ontario,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 https://t.co/y48NP0yzgn,Twitter for Android
🇨🇦 Canada,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. It's a tribute to the grandparents who died of COVID 19 and the grandc… https://t.co/wnXbbyoCe2,Twitter Web App
New York City,2020-08-29 19:44:14,"More than 1,200 students test positive for #COVID19 at major university - ABC News https://t.co/6aNhSiF5gh",Twitter for iPhone
"Aliwal North, South Africa",2020-08-29 19:44:08,I stop when I see a Stop @SABCNews @IzindabaNews24 @dailysunsa @eNCA @AliwalNorth @Ambitiouz_Ent @Official_SABC1… https://t.co/ReG8pwZoN7,Twitter for Android


In [49]:
# definindo a fonte texto tweets
tweets_covid_source <- VectorSource(tweets_covid$text)
# transformando em corpus texto tweets
tweets_covid_corpus <- VCorpus(tweets_covid_source)

In [50]:
print(tweets_covid_corpus)
tweets_covid_corpus[[1]]
tweets_covid_corpus[[1]]$content

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 179108


<<PlainTextDocument>>
Metadata:  7
Content:  chars: 140

[1] "If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0"

## Aplicando a limpeza e normalização dos dados em toda base Pub Med

In [61]:
remover_https_url <-function(texto)gsub("https.*","",  texto)
remover_http_url <-function(texto)gsub("http.*","",  texto)
remover_barra <-function(texto)gsub("/", "", texto)
remover_barras <-function(texto)gsub("\\|", "", texto)
remover_arroba <-function(texto)gsub("@", "", texto)
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(remover_https_url))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(remover_http_url))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(remover_barra))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(remover_barras))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(remover_arroba))                                                        
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(removePunctuation))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(removeNumbers))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, content_transformer(stripWhitespace))
tweets_covid_corpus <-tm_map(tweets_covid_corpus, removeWords, words = c(new_stops))
tweets_covid_corpus[[1]]$content

[1] "If I smelled  scent  hand sanitizers today  someone   past I  think    intoxicated … "

In [65]:
#Term Frequency - Document Frequency matriz
tweets_covid_dtm <- DocumentTermMatrix(tweets_covid_corpus)
tweets_covid_dtm

<<DocumentTermMatrix (documents: 179108, terms: 157092)>>
Non-/sparse entries: 1708840/28134725096
Sparsity           : 100%
Maximal term length: 78
Weighting          : term frequency (tf)

In [67]:
tweets_covid_tdm <- TermDocumentMatrix(tweets_covid_corpus)
tweets_covid_tdm

<<TermDocumentMatrix (terms: 157092, documents: 179108)>>
Non-/sparse entries: 1708840/28134725096
Sparsity           : 100%
Maximal term length: 78
Weighting          : term frequency (tf)